# Import usefull packages


In [1]:
import spotipy
import pandas as pd
import json
import pprint
import config
from itertools import *
from bs4 import BeautifulSoup as bs
import sys
import random
import requests
import pandas as pd
import pickle
from spotipy.oauth2 import SpotifyClientCredentials
sp= spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=config.client_id,client_secret=config.client_secret))

#  create big df  (empty)

In [12]:
big_df=pd.DataFrame(columns=["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","id","duration_ms"])

In [13]:
big_df

Empty DataFrame
Columns: [danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, id, duration_ms]
Index: []

# get features function

In [9]:
def get_features_df(playlist_id):
    global big_df
    list_of_audio_features=[]
    artist_name=[]
    song_name=[]
   
    results = sp.user_playlist_tracks("thesoundsofspotify",playlist_id,market="DE")
    
    tracks = results['items']
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
        
#getting features per song in  playlist 
    for value in range(0,len(tracks)):
        
        list_of_audio_features.append(sp.audio_features(tracks[value]["track"]["id"])[0])
    

#creating df with audio features and adding song name and artist     
    df=pd.DataFrame(list_of_audio_features)    
    df=df[["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","id","duration_ms"]]
    
    #df.insert(0,"song", song_name)
    #df.insert(1,"artist", artist_name)
   
    big_df =big_df.append(df)
    
    return big_df

# Scrape 6000 playlist ids 

- taken from https://everynoise.com/everynoise1d.cgi?scope=all

In [2]:
url = "https://everynoise.com/everynoise1d.cgi?root=alt%20z&scope=all"
response = requests.get(url)
response.status_code


200

In [3]:
soup = bs(response.content, "html.parser")

In [4]:
a = soup.select("table tr td .note")
playlist_list = [str(i['href']) for i in a]
id_list = [i.split(':')[3] for i in playlist_list]
id_list


['6TySxsLwRVYqEh7LZ6fyq8',
 '0OGpuF8rnCzlcjJLsdywny',
 '1BYospE2cLB28m3i84dcTV',
 '6gS3HhOiI17QNojjPuPzqc',
 '5nUyv9wyZKjdUW1rk3smTo',
 '7nHgmv7uyIA1KHj6qTttjH',
 '0lXSqruCjvX1e7hw2KGvKw',
 '41997GIfsUgMnsPLFiuLeS',
 '1aYiM4zLmBuFq0Fg6NQb6a',
 '10FCW9lj0NdeoYI5VVvVtY',
 '6TGOKSobwpEoOtx80edFyV',
 '67qIuPZRX12l7yMpxBQpxs',
 '29qGA62efPsgLp6kblUPSg',
 '2ZIRxkFuqNPMnlY7vL54uK',
 '2QYND559WzeB2urcTaOG0j',
 '6rZGKd5ndSUnGeJKuBhUej',
 '2XXreCGINlOqm8wEn7WszA',
 '0ShRnc2cyEyxkLbtZC4Yim',
 '5kdP7VQedYdDzhTvfovu0u',
 '4CfyHGpt8qPlLXlstcE2Wh',
 '1Rn01P3YPioy5QRQoo32Lv',
 '0yVmsZK4AtiNMpby2WqLZY',
 '7mBz3DA4Lt7Bsbtr5TxklP',
 '7AqOmLV0JV4a3GpIU1qxbL',
 '5yagbqWY6gDlqB893zjOmO',
 '1HGdDFrmuZ9cfLJwJ7Eb3s',
 '0FkLYgMF09lmzytexAraKv',
 '47bikoA9qfXDT5VVhzsjy9',
 '0hkXwJmcwu7hABTyspKI9b',
 '5z1A6hdw1MJrmZzDusE4Q6',
 '08RIk2wKqMNCSzEaZCcHhi',
 '6ZbHMh1injVPBZTXnDKnYp',
 '3ejnp6KkIXiZBWsCzujQ6K',
 '78IYTCUA99H1rwKXjTYSCO',
 '49KNl5r9iJkG8BYo8VlWxX',
 '2BeLGTqy54PhX1YT1jTts9',
 '5VnHdgsAhzWNkd29Rv9ErH',
 

In [5]:
len(id_list)

5962

# Creating chunks of 250 playlist each

In [10]:
chunk_size= 200
result=[id_list[i:i + chunk_size] for i in range(0, len(id_list), chunk_size)]
print(result)

[['6TySxsLwRVYqEh7LZ6fyq8', '0OGpuF8rnCzlcjJLsdywny', '1BYospE2cLB28m3i84dcTV', '6gS3HhOiI17QNojjPuPzqc', '5nUyv9wyZKjdUW1rk3smTo', '7nHgmv7uyIA1KHj6qTttjH', '0lXSqruCjvX1e7hw2KGvKw', '41997GIfsUgMnsPLFiuLeS', '1aYiM4zLmBuFq0Fg6NQb6a', '10FCW9lj0NdeoYI5VVvVtY', '6TGOKSobwpEoOtx80edFyV', '67qIuPZRX12l7yMpxBQpxs', '29qGA62efPsgLp6kblUPSg', '2ZIRxkFuqNPMnlY7vL54uK', '2QYND559WzeB2urcTaOG0j', '6rZGKd5ndSUnGeJKuBhUej', '2XXreCGINlOqm8wEn7WszA', '0ShRnc2cyEyxkLbtZC4Yim', '5kdP7VQedYdDzhTvfovu0u', '4CfyHGpt8qPlLXlstcE2Wh', '1Rn01P3YPioy5QRQoo32Lv', '0yVmsZK4AtiNMpby2WqLZY', '7mBz3DA4Lt7Bsbtr5TxklP', '7AqOmLV0JV4a3GpIU1qxbL', '5yagbqWY6gDlqB893zjOmO', '1HGdDFrmuZ9cfLJwJ7Eb3s', '0FkLYgMF09lmzytexAraKv', '47bikoA9qfXDT5VVhzsjy9', '0hkXwJmcwu7hABTyspKI9b', '5z1A6hdw1MJrmZzDusE4Q6', '08RIk2wKqMNCSzEaZCcHhi', '6ZbHMh1injVPBZTXnDKnYp', '3ejnp6KkIXiZBWsCzujQ6K', '78IYTCUA99H1rwKXjTYSCO', '49KNl5r9iJkG8BYo8VlWxX', '2BeLGTqy54PhX1YT1jTts9', '5VnHdgsAhzWNkd29Rv9ErH', '1L1Hmf2PkeZbiiQbgD93hC', '3TSPcCQfH

In [11]:
len(result)

30

- chunks completely scraped 
- 29. 
- 28.
- 26.


## We can add 30 times 250 playlist to data frame

In [17]:
for i in  result[0][10:20]:
        print(i)
        get_features_df(i)

6TGOKSobwpEoOtx80edFyV


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_22382/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


67qIuPZRX12l7yMpxBQpxs


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_22382/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


29qGA62efPsgLp6kblUPSg


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_22382/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


5kdP7VQedYdDzhTvfovu0u


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_22382/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


4CfyHGpt8qPlLXlstcE2Wh


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_22382/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


In [ ]:
for i in  result[3]:
        print(i)
        get_features_df(i)

In [15]:
big_df

danceability energy loudness speechiness acousticness instrumentalness  \
0         0.705  0.637   -7.367       0.084        0.126           0.0308   
1         0.562    0.9    -5.41      0.0403        0.287         0.000819   
2         0.863  0.532   -8.709      0.0344        0.669         0.000158   
3         0.523  0.698   -7.598      0.0354        0.329         0.000019   
4         0.566   0.42  -11.168      0.0345        0.859              0.0   
..          ...    ...      ...         ...          ...              ...   
76        0.827  0.523   -8.121      0.0459       0.0751          0.00291   
77          0.8  0.843   -7.257        0.25      0.00296              0.0   
78        0.661  0.745    -4.29      0.0292       0.0154              0.0   
79        0.663  0.884   -7.521        0.25         0.17              0.0   
80        0.762  0.508   -7.927      0.0441     0.000306            0.854   

   liveness valence    tempo                      id duration_ms  
0     0.233   0.819  127.656  0wNf3Yk11625KJ6mDkEjzu      185305  
1     0.336   0.693  138.046  5NUFanXmuh51dgRQ18xRAQ      163478  
2     0.224   0.868  113.063  6nwuEYRXKrrq9Ag4S7fzmh      181776  
3    0.0846   0.307  124.079  0DnZgTbQaUps8KG08ykZYp      257057  
4     0.111   0.199    79.97  6t0oAanLFGaERBbL0Jit4S      223817  
..      ...     ...      ...                     ...         ...  
76    0.113   0.142   124.01  2BjbG541dRwGhaketIPEXI      236462  
77    0.313   0.837  102.553  21iT3SqTPnWRGr49doXSlr      161098  
78    0.131    0.46  100.732  0EQsvl1H51El8ZCnYbSsSp      181071  
79    0.103   0.911  160.083  78Jpy7RTri3JulwgQ9sNEP      196687  
80    0.145   0.168  130.004  0sYLS9g61uh9VoADfOpnpe      273202  

[5771 rows x 11 columns]

In [241]:
for item in range(7,8):
    for i in  result[0]:
        print(i)
        get_features_df(i)

6TySxsLwRVYqEh7LZ6fyq8


KeyboardInterrupt: 

In [45]:
for item in range(8,9):
    for i in  result[0]:
        print(i) 
        get_features_df(i)

6TySxsLwRVYqEh7LZ6fyq8


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


0OGpuF8rnCzlcjJLsdywny


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


1BYospE2cLB28m3i84dcTV


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


1HGdDFrmuZ9cfLJwJ7Eb3s


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


0FkLYgMF09lmzytexAraKv


TypeError: 'NoneType' object is not subscriptable

In [50]:
for item in range(9,10):
    for i in  result[0]:
        print(i)
        get_features_df(i)

6TySxsLwRVYqEh7LZ6fyq8


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


0OGpuF8rnCzlcjJLsdywny


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


1BYospE2cLB28m3i84dcTV


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


1HGdDFrmuZ9cfLJwJ7Eb3s


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


0FkLYgMF09lmzytexAraKv


TypeError: 'NoneType' object is not subscriptable

In [230]:
for i  in  result[10][100:250]:
    print(i)
    get_features_df(i)

2TX3gfOXHHoHZiMLK0SVmq


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


52qm1kdbyMhYglyJTzEakf


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


6CXOnBtD2z6rNI9G6APj8X


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


4PRTrvutXqjv9Ec5CVO7w6


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


4XXr357Jej7eUBh7XPK8hb


KeyboardInterrupt: 

In [227]:
for i  in  result[11]:
    print(i)
    get_features_df(i)

7AWU30giFff2HhHrApuFGz


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [245]:
big_df

Empty DataFrame
Columns: [danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, id, duration_ms]
Index: []

In [66]:
for i  in  result[12]:
    print(i)
    get_features_df(i)

7dowgSWOmvdpwNkGFMUs6e


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


7qshObBpTeNY6nyMtcXXCw


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


0tqAfBZxbPFjeWwKFpfH9F


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


7HAvXnynEl18Ay6mabzRRF


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


5pkSkAC6JTpIgg0zSQoWdF


AttributeError: 'NoneType' object has no attribute 'keys'

In [67]:
for i  in  result[13]:
    print(i)
    get_features_df(i)

4QO5FPI0JxcUzJhGU4i8rI


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


54kgZCCHhUPnQWU0T8aHb8


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


5iKvDdecBJSQ5Qg46MN135


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


3B7t8HFv1PowgvPsLq6Qbf


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


18rkg4zW2Xuo6RQCZjoq10


TypeError: 'NoneType' object is not subscriptable

In [68]:
for i  in  result[14]:
    print(i)
    get_features_df(i)

76EUKJvEF2zPCLt0AuPvXT


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


3oBf8kc02esfPC3PUuSZks


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


04oO8fvjG3alSM9OcEfT3y


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


63XcNlx6B8Ilew6hsGPr82


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


7lqKESEBuOaNuquLaHdKrO


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [73]:
for i  in  result[15]:
    print(i)
    get_features_df(i)

0gdPJDXUHuVod0WX0DEZpx


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


5ggt9QATTOmCuRAPV3r7QF


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


7HSFHkTHpBbDuhTyP68zww


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


0o1QLc0vIBELSqrIEPXqQg


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


5YLwbQ51ZpX888YWodZpv2


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [75]:
for i  in  result[16]:
    print(i)
    get_features_df(i)

13rgRSFVuyKQ0YtrZDJD6R


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


7xVNaEai3Tk9uOZ2p3sdg5


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


3uwJeZkSKE219OiTdOV0b0


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


2UITkS3qkO0x6bKVzKZrbS


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


0HzeEyoLQcd9KCfda4yBoA


TypeError: 'NoneType' object is not subscriptable

In [74]:
big_df

danceability energy loudness speechiness acousticness instrumentalness  \
0         0.752  0.698    -6.53       0.306         0.52              0.0   
1         0.832  0.738   -4.417      0.0783        0.258         0.000015   
2         0.603  0.512   -6.687      0.0402        0.454          0.00014   
3         0.321  0.705    -5.41      0.0943         0.17              0.0   
4         0.477  0.128  -14.567      0.0707        0.928              0.0   
..          ...    ...      ...         ...          ...              ...   
79        0.701  0.705   -8.361       0.115        0.542         0.000199   
80        0.529  0.994   -1.947       0.194       0.0455         0.000017   
81        0.523  0.681   -5.851       0.108        0.623              0.0   
82        0.718  0.765   -5.164      0.0566       0.0745              0.0   
83        0.615  0.929    -1.56       0.161        0.333              0.0   

   liveness valence    tempo                      id duration_ms  
0     0.211   0.347  132.058  0g21KZ1XJuhwexWPLpuEt1      159453  
1      0.22    0.49  127.874  3fMNMAP51H7NywqzoukIRy      133876  
2     0.154   0.271   145.91  4xb8JHIN56xixSl6vkSPjm      200267  
3    0.0514   0.302   62.013  0O8feNuKk5rTTip2bTyOoC      184923  
4    0.0875    0.38   88.917  51xRMJFRBrW9C1RI1qW4S0      238576  
..      ...     ...      ...                     ...         ...  
79    0.103    0.76   89.984  6Yx4ke3NE26JcEoWcG36IK      216024  
80    0.319   0.506   74.996  3RU6Hh09oX8QgsWhmRtTli      285753  
81    0.122   0.451   82.385  4brO1SuRFMwOutegq7BxsY      235747  
82    0.296   0.809   84.868  1M6JfXfpjHgy1bbQyrXzzm      317500  
83     0.11   0.641   95.179  6A77JwLaTUuiRqz4f9VZiO      256004  

[124210 rows x 11 columns]

In [76]:
for i  in  result[17]:
    print(i)
    get_features_df(i)

5jJRdxN4pD29Uhj0ZIRFPf


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


14R3X4pqwNcREUaHLR8ijq


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


6UL8UnDWu9c95luN7S4wRs


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


3Qz6ukyPREhIPinAnPsAIW


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


19sgQqGAcGplazJZTXeLUG


TypeError: 'NoneType' object is not subscriptable

In [78]:
big_df

danceability energy loudness speechiness acousticness instrumentalness  \
0          0.752  0.698    -6.53       0.306         0.52              0.0   
1          0.832  0.738   -4.417      0.0783        0.258         0.000015   
2          0.603  0.512   -6.687      0.0402        0.454          0.00014   
3          0.321  0.705    -5.41      0.0943         0.17              0.0   
4          0.477  0.128  -14.567      0.0707        0.928              0.0   
..           ...    ...      ...         ...          ...              ...   
103         0.85  0.516   -8.291      0.0665         0.28            0.751   
104        0.624  0.291  -20.189       0.044        0.763            0.928   
105        0.505  0.897   -6.459      0.0385      0.00704            0.888   
106        0.596  0.786   -6.701      0.0318       0.0103            0.891   
107        0.546   0.74   -8.498      0.0322       0.0192            0.795   

    liveness valence    tempo                      id duration_ms  
0      0.211   0.347  132.058  0g21KZ1XJuhwexWPLpuEt1      159453  
1       0.22    0.49  127.874  3fMNMAP51H7NywqzoukIRy      133876  
2      0.154   0.271   145.91  4xb8JHIN56xixSl6vkSPjm      200267  
3     0.0514   0.302   62.013  0O8feNuKk5rTTip2bTyOoC      184923  
4     0.0875    0.38   88.917  51xRMJFRBrW9C1RI1qW4S0      238576  
..       ...     ...      ...                     ...         ...  
103   0.0454   0.514   93.274  027PEN37zhx4CKCdF4ZqPG      273507  
104     0.11   0.341  119.816  5X1hhT5YLeeniK9fysUozo      264208  
105   0.0908   0.579  143.995  0w0p3j0YBIj68bkInrGoJM      315773  
106    0.126   0.587  130.058  4KyCwtnIlqHo4SgQZRCIpT      296279  
107    0.106   0.319  132.034  0BAkyTlT3lrcIlYx0cLzYx      544559  

[129613 rows x 11 columns]

In [79]:
for i  in  result[18]:
    print(i)
    get_features_df(i)

2b3npHUEdwZfeLXj8JR1dh


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


6dvgLyeXG3HLGqtMa8wAX0


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


28OtIBYwNTMWc1P3HIizQH


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


7d9YLozBOYeouQ0UyKPUpZ


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


5yBIfrNoe8gDDRkddgTm4d


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [84]:
for i  in  result[19]:
    print(i)
    get_features_df(i)

4Svlg7olD7nlAaE9j93K5m


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


2QVsIulY60RZs8u18nPccL


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


2Wh7WpNxoec6eoJXv7QNDk


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


6FS4N9H4Kc4VSNGiK8vZPH


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


0yw5Q26eYIvaBediIFrYwN


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [85]:
for i  in  result[20]:
    print(i)
    get_features_df(i)

520PdmxBhzhsOIfR0lAMth


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


2Dsz5JxEeRz0nAC0plSEMF


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


46bmK7CFBoQjguGbziZgub


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


2SMDVBkKqnZL9AoUWaZ2qs


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


5OzAgYmdiqJKWjGvX7cP4Q


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [89]:
for i  in  result[21]:
    print(i)
    get_features_df(i)

2xzvhh7pEpGPJOLSE3FcTo


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


18b0DBiqVtRuj34OKPCgj6


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


7B3rUVfCVpOmeezMZ5cJEk


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


1cokzRjm0SxgXXjQaBPtEa


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


7yKED2Klt77liZOLVAwzma


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [92]:
for i  in  result[22]:
    print(i)
    get_features_df(i)

4g8jY2NYAlj7sBdFHtrgdF


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


2QeOCpMQDMK4ZHfqT2pIc3


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


2XFrsvPGbamN9w2C57a9hg


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


7FAkb5HaFNBqFFVVGFvMTE


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


6wKQSVUwtqy0SxtBEGQcDs


TypeError: 'NoneType' object is not subscriptable

In [94]:
for i  in  result[23]:
    print(i)
    get_features_df(i)

6FbDQuTcGT0IoZCtsbCISv


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


6pt4hCDsGV4O1xEBGhaiLI


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


78zBtTmgkplb8MUI3Fijek


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


6rUKKXL8dj1grd1jrmpZ3G


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


5EyFMotmvSfDAZ4hSdKrbx


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [96]:
for i  in  result[24]:
    print(i)
    get_features_df(i)

0UYBXYXU6iAs0RJqnteZW7


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


2c18XJtmWR962C1qAM6HAN


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


2dzVUieQEga2NiLQT4uTR2


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


5DcksqG6G4xJ9nu1t15wEQ


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


3EP7fmtL45NXEL0tpoO2ca


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [100]:
for i  in  result[25]:
    print(i)
    get_features_df(i)

66vH732A5RHBhtfEZHbxa1


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


4pJ4ZEzj93WnpU4CUgVxH1


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


5bNJ8LeBiHjPcsmumc9i3P


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


1vop8KpVahupGKRDvAWDrr


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


0VKu9BTDhgTrFjfBxsJ7PN


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [206]:
for i  in  result[26][113:]:
    print(i)
    get_features_df(i)

7jJ2VqsOJsd2hmsHLSOedD


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


23TcHEHll7Ds78hkaiohPx


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


19VC4dfHWpoxgPGLQtqLxN


...

/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


5cU8GY8cqeSTkgTXaYzo5z


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


57FYqlZy1HQtyNT739WL67


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [219]:
result[2][21:]

['63NyygAMIKSxy6LRG0eT5i',
 '39DfjCVzK3pnszkIPeXN17',
 '1QdpuPWvKE55QcNJZWAjSU',
 '2xCIMhNgOWQm1iWpOdg0w3',
 '196T2qP29EjWxI1x6oSiXa',
 '5NmSX3apChYsQj54LLufIV',
 '40jbkOIWkM57FnXTokMzU9',
 '6YcVrEYohgdJDds0SJ70CM',
 '3wTmRDseZi0s4Q0OpBUg4q',
 '6UoFptna0FeYqzMeOR8SVr',
 '4zC9gA0aRcK5KP0hANQmFy',
 '0wYpNflDnwSC3OZqdrQCBt',
 '3UhutiIcXo23ICdnsxBEFE',
 '1YlQnXnGXy3oUayr7CunWv',
 '5eXtMliOlZWHMcDDRkANhL',
 '72nKBOJxfLUr3iuUUMn2nV',
 '7MFj9QO8OjHAULmCOlredr',
 '1VnqViluZQhxqcgWN1R8mR',
 '0Ud1B5OYxleiuspbBXZI2t',
 '4APG0kwkbEt77VxOuqKCUl',
 '1lMgl0D1HZVvtq16pB13ew',
 '6N8qENHslBLpLYVEOP9Ebw',
 '70f3S3d60uyOdssfCZ9AEe',
 '29W4FPNZsBNek3jA5zKfBh',
 '4HxnIDtqnU3BPh4yVWZg2b',
 '2i2T2aO3SnyzstYOrLALW7',
 '1nJv9YpODF9VdIg4zpHPGl',
 '1Ph8pxls7GNLTFH7fcWxKh',
 '19TTQtvqRkqHiIwxdHCbhS',
 '1C3oYxpDKj4OHf4sB2gQQ5',
 '7eGVwau0N6LFe0tDGSgnfE',
 '1rlHnACGatHAOQrZ3WjEI3',
 '6EC3lo1QkeoX9u0lV3NpWh',
 '2Kig10nWiTUVz0WxAaUF3d',
 '59qMoqvlNat6Knc0feSWJn',
 '1u5mmyg0iXNefdvJnyYxR1',
 '2b5xbXrNNeqxpcqSpy7JLC',
 

In [220]:
for i  in  result[2][21:]:
    print(i)
    get_features_df(i)

63NyygAMIKSxy6LRG0eT5i


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


39DfjCVzK3pnszkIPeXN17


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


1QdpuPWvKE55QcNJZWAjSU


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


2xCIMhNgOWQm1iWpOdg0w3


/var/folders/1n/phyrz16s0rd1_mpv046jgqf80000gn/T/ipykernel_17629/3181457209.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df =big_df.append(df)


196T2qP29EjWxI1x6oSiXa


KeyboardInterrupt: 

In [225]:
big_df

danceability energy loudness speechiness acousticness instrumentalness  \
0          0.752  0.698    -6.53       0.306         0.52              0.0   
1          0.832  0.738   -4.417      0.0783        0.258         0.000015   
2          0.603  0.512   -6.687      0.0402        0.454          0.00014   
3          0.321  0.705    -5.41      0.0943         0.17              0.0   
4          0.477  0.128  -14.567      0.0707        0.928              0.0   
..           ...    ...      ...         ...          ...              ...   
489        0.676  0.545   -8.528      0.0293        0.332         0.000429   
490        0.536  0.476   -8.161      0.0323        0.184         0.000003   
491        0.592  0.708   -5.197      0.0251     0.000221              0.0   
492        0.718  0.714   -5.701      0.0451        0.419              0.0   
493        0.551  0.845   -5.258      0.0702       0.0183              0.0   

    liveness valence    tempo                      id duration_ms  
0      0.211   0.347  132.058  0g21KZ1XJuhwexWPLpuEt1      159453  
1       0.22    0.49  127.874  3fMNMAP51H7NywqzoukIRy      133876  
2      0.154   0.271   145.91  4xb8JHIN56xixSl6vkSPjm      200267  
3     0.0514   0.302   62.013  0O8feNuKk5rTTip2bTyOoC      184923  
4     0.0875    0.38   88.917  51xRMJFRBrW9C1RI1qW4S0      238576  
..       ...     ...      ...                     ...         ...  
489    0.124   0.589  143.857  0NLRVZREht65Ga5WmlrDPE      197878  
490    0.104   0.496   113.09  5RjZ13Ce3dRUXRBR7lbfTL      195400  
491    0.174   0.219  103.061  1GIcsaFqm5xn5mkl97v0lV      188637  
492    0.284   0.928   97.197  1tsxqGAS9dZh3GYlJUmpkk      274040  
493    0.158   0.382  148.109  5z4OlVj60Fqxc9PXWgg1rb      177266  

[273115 rows x 11 columns]

In [18]:
big_df.to_csv("addd_rows.csv",index=False)